In [ ]:
import pickle
import sys
from zoneinfo import ZoneInfo
sys.path.append("../")

from dotenv import load_dotenv
load_dotenv()
import geopandas as gpd
import importlib
import copy
import logging
import contextily as cx
import gtfs_kit as gk
import fastsim as fsim
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import lightning.pytorch as pl
import rasterio as rio
from rasterio.plot import show
import seaborn as sns
import shapely
import statsmodels.api as sm
from torch.utils.data import DataLoader

from openbustools import plotting, spatial, standardfeeds
from openbustools.traveltime import data_loader, model_utils
from openbustools.drivecycle import trajectory
from openbustools.drivecycle.physics import conditions, energy, vehicle

In [ ]:
epsg = 32148
timezone = "America/Los_Angeles"
coord_ref_center = [386910,69022]
chop_n = 500
point_sep_m = 300
dem_file = Path("..","data","kcm_spatial","usgs10m_dem_32148.tif")
phone_trajectory_folders = [x for x in Path("..","data","kcm_sensor","match_realtime","gnss_validated").glob("*") if x.is_dir()]
realtime_folder = Path("..","data","kcm_sensor_realtime")
gnss_solution_file = Path("..","data","gnss","CEErover_solution_20240312160029.LLH")

model_folder = "../logs"
run_label = "/kcm"
model_type = "GRU"
fold_num = 0
version = "version_3"
model = model_utils.load_model(model_folder, run_label, model_type, fold_num, version=version)
model.eval()

### Expand Predicted to Full Network

In [ ]:
# Load a static feed and break each shape into regularly spaced points
static_file = Path("..", "data", "kcm_static", "2023_03_18")
static_feed = gk.read_feed(static_file, dist_units="km")
route_shape_points = standardfeeds.segmentize_route_shapes(static_feed, epsg=epsg, point_sep_m=point_sep_m)
route_shape_points

In [ ]:
# Turning regularly spaced points w/speeds into calculated distances and times
# Important to base distance on sequence id because some points do not have observations
for k,df in plot_df_groups.items():
    plot_df_groups[k]['cumul_dist_m'] = df['seq_id'] * point_sep_m
    plot_df_groups[k]['calc_dist_m'] = df['cumul_dist_m'].diff().fillna(0)
    plot_df_groups[k]['calc_time_s'] = df['calc_dist_m'] / df['speed_mean']
    plot_df_groups[k]['cumul_time_s'] = df['calc_time_s'].cumsum()
    plot_df_groups[k]['speed_std'] = df['speed_std'].bfill().ffill()

In [ ]:
# Create trajectory for each shape
plot_trajectories = []
for shape_id, df in plot_df_groups.items():
    traj = trajectory.Trajectory(
        point_attr={
            "lon": df.to_crs(4326).geometry.x.to_numpy(),
            "lat": df.to_crs(4326).geometry.y.to_numpy(),
            "locationtime": df.cumul_time_s.to_numpy(),
            "measured_speed_m_s": df.speed_mean.to_numpy(),
            "seq_id": df.seq_id.to_numpy(),
            "count_n": df.count_n.to_numpy()
        },
        traj_attr={
            'shape_id': shape_id,
            "coord_ref_center": coord_ref_center,
            "epsg": epsg,
            "dem_file": dem_file,
            "t_min_of_day": 9*60,
            "t_day_of_week": 4,
            "start_epoch": df.cumul_time_s.iloc[0],
            "end_epoch": df.cumul_time_s.iloc[-1]
        },
        resample=False
    )
    plot_trajectories.append(traj)

In [ ]:
# Predict speeds along each trajectory
importlib.reload(trajectory)
preds = trajectory.predict_speeds(plot_trajectories, model)[0]
preds['mask'][0,:] = True
pred_times = [preds['preds_seq'][:,i][preds['mask'][:,i]] for i in range(preds['preds_seq'].shape[1])]
dists = [traj.gdf['calc_dist_m'].to_numpy() for traj in plot_trajectories]
pred_speeds = [dists[i] / pred_times[i] for i in range(len(pred_times))]

for traj in plot_trajectories:
    traj.gdf['pred_speed_m_s'] = pred_speeds.pop(0)

In [ ]:
energy_results = []
efficiencies = []

veh = fsim.vehicle.Vehicle.from_vehdb(63, veh_file=Path("..", "data", "FASTSim_py_veh_db.csv")) # New Flyer XE40
print(f"Vehicle: {veh.scenario_name}\n")

for i, traj in enumerate(plot_trajectories):
    # Energy analysis for aggregated
    cycle_agg = {
        "cycGrade": np.clip(spatial.divide_fwd_back_fill(np.diff(traj.gdf['calc_elev_m'], prepend=traj.gdf['calc_elev_m'].iloc[0]), traj.gdf['calc_dist_m']), -0.15, 0.15),
        "mps": spatial.apply_sg_filter(traj.gdf["calc_speed_m_s"].to_numpy(), 8, 0, 30),
        "time_s": traj.gdf['cumul_time_s'].to_numpy(),
        "road_type": np.zeros(len(traj.gdf))
    }
    cycle_agg = fsim.cycle.Cycle.from_dict(fsim.cycle.resample(cycle_agg, new_dt=1))
    sim_drive_agg = fsim.simdrive.SimDrive(cycle_agg, veh)
    sim_drive_agg.sim_drive()
    # Energy analysis for predicted
    cycle_pred = {
        "cycGrade": np.clip(spatial.divide_fwd_back_fill(np.diff(traj.gdf['calc_elev_m'], prepend=traj.gdf['calc_elev_m'].iloc[0]), traj.gdf['calc_dist_m']), -0.15, 0.15),
        "mps": spatial.apply_sg_filter(traj.gdf["pred_speed_m_s"].to_numpy(), 8, 0, 30),
        "time_s": traj.gdf['cumul_time_s'].to_numpy(),
        "road_type": np.zeros(len(traj.gdf))
    }
    cycle_pred = fsim.cycle.Cycle.from_dict(fsim.cycle.resample(cycle_pred, new_dt=1))
    sim_drive_pred = fsim.simdrive.SimDrive(cycle_pred, veh)
    sim_drive_pred.sim_drive()
    energy_results.append({"agg": (cycle_agg, sim_drive_agg), "pred": (cycle_pred, sim_drive_pred)})
    efficiencies.append((1 / sim_drive_agg.electric_kwh_per_mi, 1 / sim_drive_pred.electric_kwh_per_mi))

print(f"Efficiencies:")
print(f"Avg. Agg: {np.mean([x[0] for x in efficiencies]):.2f} mi/kWh")
print(f"Avg. Pred: {np.mean([x[1] for x in efficiencies]):.2f} mi/kWh\n")
print(f"RMSE Agg/Pred: {np.sqrt(np.mean([(x-y)**2 for x,y in efficiencies])):.2f} mi/kWh")

### Expand Predicted to All Networks